# Sistema de Gerenciamento de Coleta de Lixo e Controle de Zoonoses

## Contexto
O bairro **Lagoa Cheia**, localizado em uma cidade do Cariri cearense, enfrenta desafios relacionados à coleta de lixo precária e à presença de animais de rua. Esses problemas afetam diretamente a qualidade de vida dos moradores, causando alagamentos, aumento de doenças e danos às vias públicas. Este projeto visa implementar um sistema integrado para:

1. **Coleta periódica de lixo** com otimização do uso de caminhões coletores e recursos humanos.
2. **Controle de zoonoses**, com a identificação e recolhimento de animais de rua, minimizando os impactos causados.

---

## Especificações do Sistema

### Coleta de Lixo
- **Ciclos de Coleta**: Coleta ocorre a cada dois dias, com caminhões passando por pontos de coleta específicos.
- **Capacidade do Caminhão**: 400 m³, com até 3 compactações para otimizar o volume transportado.
- **Funcionários**: Cada caminhão opera com 3 a 5 funcionários, além do motorista.
- **Restrições de Tempo**: Todo o lixo deve ser coletado em **até 8 horas** por dia.

### Animais de Rua
- **Tipos de Animais**: Ratos, gatos e cachorros presentes em pontos de coleta.
- **Movimentação**:
  - Gatos espantam ratos.
  - Cachorros espantam gatos.
  - Em pontos com os três tipos, gatos e ratos fogem para locais vizinhos.
- **Controle de Zoonoses**:
  - Equipes de zoonoses recolhem gatos e cachorros usando carrocinhas com capacidade para 5 animais.
  - As carrocinhas percorrem pontos de coleta e retornam ao centro de zoonoses quando cheias.

---

## Entrada do Sistema
1. **Pontos de Coleta**:
   - Identificação única.
   - Quantidade inicial de lixo (m³).
   - Conexões com outros pontos (arestas com custo de deslocamento).
2. **Conexões**: Medem o custo de deslocamento entre dois pontos e incluem dificuldades como buracos.
3. **Animais**: Probabilidades iniciais para presença:
   - Ratos: 50%.
   - Gatos: 25%.
   - Cachorros: 10%.

### Formato do Arquivo de Entrada
```plaintext
ID_Ponto, Quantidade_Lixo, [(Destino, Custo), ...]
```

- Cada linha representa um ponto de coleta.
- `id_ponto`: Identificador único do ponto de coleta.
- `lixo_inicial`: Quantidade inicial de lixo no ponto, em metros cúbicos.
- `[(id_destino,custo)]`: Lista de conexões para outros pontos de coleta. Cada conexão tem o `id_destino` e o `custo` de deslocamento em minutos.

### Saída do Programa

A saída do programa deve indicar:
1. O número mínimo de caminhões de lixo necessários para realizar a coleta dentro do limite de 8 horas.
2. O número total de funcionários necessários para operar os caminhões.
3. O número total de animas resgatados.

### Observações

- A implementação deve garantir que o volume total de lixo seja coletado dentro do tempo estipulado.
- Caso o tempo exceda 8 horas, deve-se ajustar a configuração de caminhões e funcionários.
- A movimentação e coleta de animais pela carrocinha foi registrada e reportada ao final da simulação.

### Instruções executar o programa

- Execute a celula de geração do csv, que contem a função ```gerar_conexoes```.
- Execute a celula geral do programa, que contem o resto do codigo.


In [57]:
import csv
import random
import pandas as pd

def gerar_conexoes(num_pontos):
    conexoes = []
    for i in range(num_pontos):
        num_conexoes = random.randint(1, 4)  # Cada ponto tem entre 1 e 4 conexões
        conexoes_ponto = []
        while len(conexoes_ponto) < num_conexoes:
            destino = random.randint(0, num_pontos - 1)
            if destino != i and destino not in [c[0] for c in conexoes_ponto]:
                custo = random.randint(5, 50)  # Custo entre 5 e 50
                conexoes_ponto.append((destino, custo))
        conexoes.append(conexoes_ponto)
    return conexoes

def gerar_csv(nome_arquivo, num_pontos):
    conexoes = gerar_conexoes(num_pontos)
    with open(nome_arquivo, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        for i in range(num_pontos):
            lixo = random.randint(50, 200)  # Lixo entre 50 e 200 metros cúbicos
            writer.writerow([i, lixo, str(conexoes[i])])

# Gera o CSV com 100 pontos
gerar_csv("entrada.csv", 100)


In [58]:
import csv
import random
from collections import deque


class PontoColeta:
    def __init__(self, id, lixo, conexoes):
        self.id = id
        self.lixo = lixo
        self.conexoes = conexoes  # [(destino, custo)]
        self.animais = {"ratos": 0, "gatos": 0, "cachorros": 0}

    def inicializar_animais(self):
        if random.random() < 0.5:
            self.animais["ratos"] = random.randint(1, 5)
        if random.random() < 0.25:
            self.animais["gatos"] = random.randint(1, 3)
        if random.random() < 0.1:
            self.animais["cachorros"] = random.randint(1, 2)

    def atualizar_movimento_animais(self, pontos):
        if self.animais["gatos"] > 0 and self.animais["ratos"] > 0:
            ratos_que_fogem = max(0, self.animais["ratos"] - 1)  # Um rato é pego
            self.animais["ratos"] -= ratos_que_fogem
            self.distribuir_animais("ratos", ratos_que_fogem, pontos)

        if self.animais["cachorros"] > 0 and self.animais["gatos"] > 0:
            gatos_que_fogem = max(0, self.animais["gatos"] - 1)  # Um gato é pego
            self.animais["gatos"] -= gatos_que_fogem
            self.distribuir_animais("gatos", gatos_que_fogem, pontos)

        if (
            self.animais["ratos"] > 0
            and self.animais["gatos"] > 0
            and self.animais["cachorros"] > 0
        ):
            self.distribuir_animais("ratos", self.animais["ratos"], pontos)
            self.distribuir_animais("gatos", self.animais["gatos"], pontos)
            self.animais["ratos"] = 0
            self.animais["gatos"] = 0

    def distribuir_animais(self, tipo, quantidade, pontos):
        destinos = [dest for dest, _ in self.conexoes]
        for _ in range(quantidade):
            if destinos:
                destino = random.choice(destinos)
                pontos[destino].animais[tipo] += 1


class CarroLixo:
    def __init__(self, funcionarios, aterro):
        self.funcionarios = funcionarios # De 3 a 5 funcionarios
        self.capacidade = 400  # Capacidade inicial
        self.capacidade_atual = self.capacidade
        self.compactacoes = 0  # Contador de compactações
        self.lixo_coletado = 0
        self.aterro = aterro  # Local do aterro sanitário
        self.tempo_total = 0  # Tempo acumulado de operação

    def compactar(self):
        if self.compactacoes < 3:
            ganho = self.capacidade_atual * (1 / 3)
            self.capacidade_atual += ganho
            self.compactacoes += 1

    def deslocar_para(self, destino, pontos):
        for conexao in pontos[self.aterro].conexoes:
            if conexao[0] == destino:
                self.tempo_total += conexao[1]  # Adiciona o custo do deslocamento

    def coletar(self, ponto):
        if ponto.lixo > 0:
            tempo_coleta = ponto.lixo // self.funcionarios
            if any(ponto.animais.values()):
                tempo_coleta *= 2

            if ponto.lixo <= self.capacidade_atual:
                self.capacidade_atual -= ponto.lixo
                self.lixo_coletado += ponto.lixo
                ponto.lixo = 0
            else:
                ponto.lixo -= self.capacidade_atual
                self.lixo_coletado += self.capacidade_atual
                self.capacidade_atual = 0

            self.tempo_total += tempo_coleta
            return tempo_coleta
        return 0

    def esvaziar_no_aterro(self, pontos):
        self.deslocar_para(self.aterro, pontos)
        self.capacidade_atual = self.capacidade
        self.compactacoes = 0
        self.lixo_coletado = 0


class Carrocinha:
    def __init__(self, centro_zoonoses):
        self.capacidade = 5
        self.animais_coletados = []
        self.centro_zoonoses = centro_zoonoses  # Local do centro de zoonoses
        self.tempo_total = 0  # Tempo acumulado de operação

    def coletar(self, ponto):
        for animal in ["gatos", "cachorros"]:
            while ponto.animais[animal] > 0 and len(self.animais_coletados) < self.capacidade:
                ponto.animais[animal] -= 1
                self.animais_coletados.append(animal)

    def deslocar_para(self, destino, pontos):
        for conexao in pontos[self.centro_zoonoses].conexoes:
            if conexao[0] == destino:
                self.tempo_total += conexao[1]  # Adiciona o custo do deslocamento

    def esvaziar_no_zoonoses(self, pontos):
        self.deslocar_para(self.centro_zoonoses, pontos)
        self.animais_coletados = []


def carregar_pontos(arquivo):
    pontos = {}
    with open(arquivo, "r") as f:
        reader = csv.reader(f)
        for linha in reader:
            id = int(linha[0])
            lixo = int(linha[1])
            conexoes = eval(linha[2])  # Lista de conexões [(destino, custo)]
            pontos[id] = PontoColeta(id, lixo, conexoes)
    return pontos


def simular_movimentacao(pontos):
    for ponto in pontos.values():
        ponto.atualizar_movimento_animais(pontos)


def gerenciar_coleta(pontos, aterro, limite_tempo=480):  # 8 horas = 480 minutos
    total_lixo = sum(p.lixo for p in pontos.values())
    caminhoes_configuracao = []  # Lista de tuplas: [(caminhão_id, número_de_funcionarios)]
    pontos_animais = {}
    i = 0
    tempo_excedido = True

    while tempo_excedido:  # Continua testando configurações enquanto o tempo for excedido
        caminhoes = []
        funcionarios_totais = 0
        tempo_restante = limite_tempo
        pontos_iniciais = list(pontos.keys())

        # Reconfigura caminhões e funcionários
        for caminhao_id, funcionarios in caminhoes_configuracao:
            caminhoes.append(CarroLixo(funcionarios, aterro))
            funcionarios_totais += funcionarios

        # Caminhões começam em locais aleatórios
        random.shuffle(pontos_iniciais)

        # Simulação com configuração atual
        for caminhao in caminhoes:
            for ponto_id in pontos_iniciais:
                if tempo_restante > 0 and total_lixo > 0:
                    caminhao.deslocar_para(ponto_id, pontos)
                    tempo_coleta = caminhao.coletar(pontos[ponto_id])
                    tempo_restante -= tempo_coleta

                    if pontos[ponto_id].animais["ratos"] > 0 or pontos[ponto_id].animais["gatos"] > 0 or pontos[ponto_id].animais["cachorros"] > 0:
                        pontos_animais[i] = (pontos[ponto_id])
                        i += 1

                    if caminhao.capacidade_atual == 0:
                        if caminhao.compactacoes < 3:
                            caminhao.compactar()
                        else:
                            caminhao.esvaziar_no_aterro(pontos)

                if tempo_restante <= 0:
                    break

        # Atualiza o lixo total restante
        total_lixo = sum(p.lixo for p in pontos.values())

        # Verifica se coletou tudo dentro do tempo
        if total_lixo == 0 and tempo_restante >= 0:
            tempo_excedido = False
        else:
            # Aumenta a configuração: preenche o último caminhão antes de adicionar um novo
            if not caminhoes_configuracao or caminhoes_configuracao[-1][1] < 5:
                # Adiciona mais um funcionário ao último caminhão
                if caminhoes_configuracao:
                    caminhoes_configuracao[-1] = (
                        caminhoes_configuracao[-1][0],
                        caminhoes_configuracao[-1][1] + 1,
                    )
                else:
                    # Primeiro caminhão com 3 funcionários
                    caminhoes_configuracao.append((0, 3))
            else:
                # Adiciona um novo caminhão com 3 funcionários
                caminhoes_configuracao.append((len(caminhoes_configuracao), 3))

    return len(caminhoes_configuracao), funcionarios_totais, pontos_animais



def gerenciar_zoonoses(pontos, centro_zoonoses, limite_tempo=480):
    carrocinhas = [Carrocinha(centro_zoonoses) for _ in range(3)]
    total_capturas = 0

    for carrocinha in carrocinhas:
        tempo_restante = limite_tempo

        for ponto_id, ponto in pontos.items():
            if tempo_restante <= 0:
                break  # Parar se o tempo exceder

            # Deslocar até o ponto
            carrocinha.deslocar_para(ponto_id, pontos)
            tempo_restante -= carrocinha.tempo_total  # Atualizar tempo restante

            # Coletar animais
            carrocinha.coletar(ponto)
            tempo_restante -= 10  # Considera 10 minutos para cada coleta

            # Se a carrocinha atingir a capacidade, esvaziar
            if len(carrocinha.animais_coletados) == carrocinha.capacidade:
                carrocinha.esvaziar_no_zoonoses(pontos)
                total_capturas += carrocinha.capacidade

        # Esvaziar a carrocinha no centro de zoonoses ao final do turno
        if carrocinha.animais_coletados:
            carrocinha.esvaziar_no_zoonoses(pontos)
            total_capturas += len(carrocinha.animais_coletados)

    return total_capturas

def exibir_resultados(pontos, caminhoes, funcionarios, capturas):
    lixo_restante = sum(ponto.lixo for ponto in pontos.values())

    dados = {
        "Métrica": [
            "Número total de caminhões utilizados", 
            "Número total de funcionários necessários",
            "Número total de animais capturados",
            "Quantidade de lixo remanescente"
        ],
        "Valor": [
            caminhoes,
            funcionarios,
            capturas,
            lixo_restante
        ]
    }

    df_principal = pd.DataFrame(dados)
    

    # Exibindo os resultados
    print("==== RELATÓRIO FINAL  ====")
    display(df_principal)
    print("==== FIM DO RELATÓRIO ====")

def simular(arquivo_csv):
    pontos = carregar_pontos(arquivo_csv)
    aterro = random.choice(list(pontos.keys()))  # Define o ponto do aterro
    zoonoses = random.choice(list(pontos.keys()))  # Define o ponto do centro de zoonoses

    for ponto in pontos.values():
        ponto.inicializar_animais()

    simular_movimentacao(pontos)

    caminhoes, funcionarios, pontos_animais = gerenciar_coleta(pontos, aterro)
    capturas = gerenciar_zoonoses(pontos_animais, zoonoses)

    exibir_resultados(pontos, caminhoes, funcionarios, capturas)


# Chamada do programa
entrada_csv = "entrada.csv"
simular(entrada_csv)


==== RELATÓRIO FINAL  ====


,Métrica,Valor
0,Número total de caminhões utilizados,5
1,Número total de funcionários necessários,24
2,Número total de animais capturados,35
3,Quantidade de lixo remanescente,0


==== FIM DO RELATÓRIO ====
